Import *.json data scraped from Amazon

In [35]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
from django.utils.encoding import smart_str, smart_unicode

In [36]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [37]:
with open('All.json') as data_file:
    data = json.load(data_file)

In [38]:
condom = pd.read_json('All.json')
testing = condom.copy()

DATA CLEASING START

In [39]:
# STAR
testing.ix[testing['STAR'] == 'None', 'STAR'] = np.NaN
testing.loc[:, 'STAR'] = [str(x)[-18:-15] for x in testing.loc[:, 'STAR']]

In [40]:
# 1_STAR
testing.loc[:, '1_STAR'] = [str(x)[:-1] for x in testing.loc[:, '1_STAR']]
testing.ix[testing['1_STAR'] == 'Non', '1_STAR'] = np.NaN
testing.loc[:, '1_STAR'] = [float(x)/100 for x in testing['1_STAR']]
# 2_STAR
testing.loc[:, '2_STAR'] = [str(x)[:-1] for x in testing.loc[:, '2_STAR']]
testing.ix[testing['2_STAR'] == 'Non', '2_STAR'] = np.NaN
testing.loc[:, '2_STAR'] = [float(x)/100 for x in testing['2_STAR']]
# 3_STAR
testing.loc[:, '3_STAR'] = [str(x)[:-1] for x in testing.loc[:, '3_STAR']]
testing.ix[testing['3_STAR'] == 'Non', '3_STAR'] = np.NaN
testing.loc[:, '3_STAR'] = [float(x)/100 for x in testing['3_STAR']]
# 4_STAR
testing.loc[:, '4_STAR'] = [str(x)[:-1] for x in testing.loc[:, '4_STAR']]
testing.ix[testing['4_STAR'] == 'Non', '4_STAR'] = np.NaN
testing.loc[:, '4_STAR'] = [float(x)/100 for x in testing['4_STAR']]
# 5_STAR
testing.loc[:, '5_STAR'] = [str(x)[:-1] for x in testing.loc[:, '5_STAR']]
testing.ix[testing['5_STAR'] == 'Non', '5_STAR'] = np.NaN
testing.loc[:, '5_STAR'] = [float(x)/100 for x in testing['5_STAR']]

In [43]:
# ORIGINAL_PRICE
testing['ORIGINAL_PRICE'] = [str(x).replace('$', '').replace(',', '') for x in testing['ORIGINAL_PRICE']]
# SALE_PRICE
testing['SALE_PRICE'] = [str(x).replace('$', '').replace(',', '') for x in testing['SALE_PRICE']]

In [44]:
# RANK_IN_CONDOM
testing.loc[:, 'RANK_IN_CONDOM']  = [str(x) for x in testing.loc[:, 'RANK_IN_CONDOM']]
testing.loc[:, 'RANK_IN_CONDOM'] = [str(x).replace('#', '').split(' ')[0]  for x in testing.loc[:, 'RANK_IN_CONDOM']]
testing.ix[testing['RANK_IN_CONDOM'] == 'None', 'RANK_IN_CONDOM'] = np.NaN
testing.loc[:, 'RANK_IN_CONDOM'] = [float(x) for x in testing.loc[:, 'RANK_IN_CONDOM']]

In [45]:
# NumCUSTOMER_REVIEW
testing.loc[:, 'NumCUSTOMER_REVIEW'] = [str(x).split(' customer review')[0].split(' ')[0] for x in testing.loc[:, 'NumCUSTOMER_REVIEW']]
testing.ix[testing['NumCUSTOMER_REVIEW'] == 'None', 'NumCUSTOMER_REVIEW'] = np.NaN
testing.loc[:, 'NumCUSTOMER_REVIEW'] = [float(x) for x in testing.loc[:, 'NumCUSTOMER_REVIEW']]

In [46]:
# NumANSWERED_QUESTION
testing.loc[:, 'NumANSWERED_QUESTION'] = [str(x).split(' customer review')[0].split(' ')[0] for x in testing.loc[:, 'NumANSWERED_QUESTION']]
testing.ix[testing['NumANSWERED_QUESTION'] == 'None', 'NumANSWERED_QUESTION'] = np.NaN
testing.loc[:, 'NumANSWERED_QUESTION'] = [float(x) for x in testing.loc[:, 'NumANSWERED_QUESTION']]

In [47]:
# DERIVED FROM 'WEIGHT'
testing.loc[:, 'WEIGHT_OUNCE'] = [str(x) for x in testing.loc[:, 'WEIGHT']]
testing.loc[:, 'WEIGHT_INCH'] = [str(x) for x in testing.loc[:, 'WEIGHT']]
# INTRODUCE 2 NEW VARIABLES
# 1. OUNCE
testing.loc[:, 'WEIGHT_OUNCE'] = testing.WEIGHT_OUNCE.str.lower().str.extract('(.+?) ounce')
# 2. INCH
testing.loc[:, 'WEIGHT_INCH'] = testing.WEIGHT_OUNCE.str.lower().str.extract('(.+?) inches')

*------------------*

In [48]:
# CREATE A NEW DATAFRAME
# a. CREATE TWO NEW LISTS
customer_review_star = testing.loc[:, 'TOP_CUSTOMER_REVIEW_STAR']
customer_review = testing.loc[:, 'TOP_CUSTOMER_REVIEW']
condom_id = [x.replace('http://www.amazon.com/dp/', '') for x in testing.loc[:, 'URL']]

In [49]:
# Unique ids
unique_id = []
for i in range(0, len(condom_id)):
    unique_id.extend([condom_id[i]] + [condom_id[i]]* (len(customer_review_star[i])-1))

In [50]:
# CUSTOMER_REVIEW_STAR
temp_customer_review_star = [';'.join(x).replace(' out of 5 stars', '').split(';') for x in customer_review_star]
new_customer_review_star = [item for sublist in temp_customer_review_star for item in sublist]
# CUSTOMER_REVIEW
temp_customer_review = [''.join(x).replace('\n\n', '$TEXT$').replace('\n', '').strip().split('$TEXT$  ') for x in customer_review]
_customer_review = [item for sublist in temp_customer_review for item in sublist]
new_customer_review = [smart_str(x) for x in _customer_review]

In [51]:
# b. CREATE A NEW DATAFRAME USING THESE TWO NEW LISTS
condom_reviews = pd.DataFrame(
                    {'STAR':new_customer_review_star,
                     'REVIEW':new_customer_review,
                     'ID': unique_id})
# 'condom_reviews' CREATED AS NEW DATAFRAME

*--------------*--------------*--------------*--------------*--------------*

In [53]:
testing.loc[:, 'TOP_CUSTOMER_REVIEW'] = [smart_str(x) for x in testing.loc[:, 'TOP_CUSTOMER_REVIEW']]
# 'testing' OUTPUT FOR FURTHER PROCESSING USING R
testing.to_csv('All.csv')

In [54]:
from django.utils.encoding import smart_str, smart_unicode
condom_reviews.loc[:, 'REVIEW'] = [smart_str(x) for x in condom_reviews.loc[:, 'REVIEW']]

In [55]:
# 'condom_reviews' OUTPUT FOR FURTHER PROCESSING USING R
condom_reviews.to_csv('All_reviews.csv')

In [34]:
testing

,1_STAR,2_STAR,3_STAR,4_STAR,5_STAR,BRAND,CATEGORY,ITEM_ALSO_BOUGHT_1,ITEM_ALSO_BOUGHT_2,ITEM_ALSO_BOUGHT_3,...,PREVIEW_IMAGE_COUNT,RANK_IN_CONDOM,SALE_PRICE,STAR,TOP_CUSTOMER_REVIEW,TOP_CUSTOMER_REVIEW_STAR,URL,WEIGHT,WEIGHT_OUNCE,WEIGHT_INCH
0,0.07,0.04,0.08,0.14,0.67,Trojan,Health & Household > Sexual Wellness > Safer S...,B00G5K7L24,B00MFHDOXQ,B0073RAL3O,...,6,1.0,NaN,4.3,"[u""\n These are great. Regular old condoms, n...","[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0073R7TWU,5.2 x 2.4 x 5.4 inches,NaN,NaN
1,0.14,0.04,0.08,0.13,0.61,Trojan,Health & Household > Sexual Wellness > Safer S...,B008UYN4QA,B00MFHDOXQ,B0073RAL3O,...,6,2.0,NaN,4.1,"[u""\n These are my favorite condoms. Excellen...","[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0073RBUZ2,5.2 x 2.4 x 5.4 inches,NaN,NaN
2,0.03,0.01,0.03,0.09,0.84,Trojan,Health & Household > Sexual Wellness > Safer S...,B00ITWY1II,B00R3UKU64,B0040VNUD4,...,6,5.0,NaN,4.7,[u'\n Everyone knows Trojan is the brand that...,"[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0083HLPLA,5.2 x 2.4 x 5.4 inches ; 5.6 ounces,5.2 x 2.4 x 5.4 inches ; 5.6,5.2 x 2.4 x 5.4
3,0.13,0.05,0.10,0.17,0.55,Durex,Health & Household > Sexual Wellness > Safer S...,B01K5S5KWE,B00U2VYOSG,B00BI0BGL8,...,5,3.0,NaN,4.0,[u'\n Absolutely the best delay condoms out t...,"[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0070YFJGO,1.8 x 5.4 x 5.4 inches,NaN,NaN
4,0.18,0.05,0.10,0.12,0.55,Trojan,Health & Household > Sexual Wellness > Safer S...,B00R3UKU6O,B0073RBUZ2,B01C3JBA7M,...,6,4.0,NaN,3.9,[u'\n I have been a big fan of Trojan condoms...,"[5.0 out of 5 stars, 1.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B008UYN4QA,5.1 x 1.9 x 5.4 inches ; 2.4 ounces,5.1 x 1.9 x 5.4 inches ; 2.4,5.1 x 1.9 x 5.4
5,0.06,0.03,0.09,0.12,0.70,Trojan,Health & Household > Sexual Wellness > Safer S...,B00LO3QX44,B00G5K7L24,B00MFHDOXQ,...,3,7.0,NaN,4.4,"[u'\n This is a great product, and at this pr...","[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B010MR7F6W,7 ounces,7,NaN
6,0.06,0.03,0.08,0.13,0.70,Trojan,Health & Household > Sexual Wellness > Safer S...,B0073RBUZ2,B0040VPKC8,B0040Y651A,...,8,6.0,NaN,4.3,"[u'\n No children yet.\n', u'\n It may be a ...","[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0073RAL3O,5.2 x 2.4 x 5.4 inches,NaN,NaN
7,0.05,0.04,0.06,0.17,0.68,LifeStyles,Health & Household > Sexual Wellness > Safer S...,B00G5K7L24,B01J939GVC,B004TTXA7I,...,3,8.0,NaN,4.4,"[u""\n My boyfriend and I had to go back to us...","[5.0 out of 5 stars, 5.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B00JKDH7WM,1.8 x 5.4 x 5.4 inches ; 4 ounces,1.8 x 5.4 x 5.4 inches ; 4,1.8 x 5.4 x 5.4
8,0.06,0.01,0.07,0.13,0.73,Trojan,Health & Household > Sexual Wellness > Safer S...,B00ITWY1II,B0064FHD5A,B0040VMCYW,...,7,9.0,NaN,4.5,"[u""\n I used to be a fan of the Thin Magnum b...","[5.0 out of 5 stars, 5.0 out of 5 stars, 4.0 o...",http://www.amazon.com/dp/B00R3UKU64,3.1 x 1.6 x 5.4 inches,NaN,NaN
9,0.08,0.03,0.07,0.12,0.70,Naturalamb,Health & Household > Sexual Wellness > Safer S...,B003LZQ0SW,B00QBP364A,B004GEYR28,...,8,11.0,NaN,4.2,"[u""\n I hate condoms. Almost as much as I ha...","[5.0 out of 5 stars, 4.0 out of 5 stars, 5.0 o...",http://www.amazon.com/dp/B0073RLP3Y,3.1 x 1.6 x 5.4 inches,NaN,NaN
